In [7]:
import pandas as pd
from configurations.general_confs import SparseMethods

In [8]:
df = pd.read_csv("./experiments/diff_sparse2/best_acc.csv", sep=";")
df["Score"] = df["Score"] * 100

In [9]:
def get_dataset(model_name):
    """ Get data set from model name. Is generally in model name for visdom. """
    for d in ["fashionmnist", "cifar10", "mnist"]:
        if d in model_name:
            return d

def sparse_method(model_name):
    """ Get sparse method from model name"""
    
    if "loss" in model_name: 
        if "loss_001" in model_name:
            sparse_name = "Entropy (0.01)"
        elif "loss_01" in model_name:
            sparse_name = "Entropy (0.1)"
        
    else: 
        # get sparse string from model_name
        sparse_str = model_name.split(":")[1]

        # init sparse methods by sparse string
        sparse = SparseMethods(sparse_str)
        
        method_list = []
        for step in sparse:
            if step['target'] == "none":
                method_list.append("none")
            else: 
                percent_list = '-'.join([str(e) for e in step['percent']])
                method_list.append(f"{step['target']} {step['method']} {percent_list}")
        
        sparse_name = " & ".join(method_list)
    
    if "no" in model_name[-2:]: 
        sparse_name += " No"
        
    return sparse_name
            
    

#### Select df sub

In [42]:
dataset = "cifar10"
sparse_method_not_contains = " No"

df["dataset"] = df["ModelName"].apply(lambda name:get_dataset(name))
df["sparse_method"] = df["ModelName"].apply(lambda x:sparse_method(x))


df_sub = df[df['dataset'] == dataset]
df_sub = df_sub[~df_sub['sparse_method'].str.contains(sparse_method_not_contains)]


#### Make nice column of df_sub

In [43]:
new_sparse_method_col = dataset.upper() # hacky solution to get nice title
df_sub = df_sub.rename(index=str, columns={"sparse_method": new_sparse_method_col})
df_sub = df_sub.groupby([new_sparse_method_col])["Score"].describe()[["count", "mean", "std"]]
df_sub = df_sub.round(2)
df_sub

,count,mean,std
CIFAR10,,,
Entropy (0.01),4.0,64.88,0.56
Entropy (0.1),4.0,61.99,2.02
edges random 0.0-0.3,4.0,64.32,0.28
edges random 0.0-0.6,4.0,64.11,0.16
edges random 0.3-0.3,4.0,63.91,0.92
edges sample 0.3-0.3,4.0,63.96,1.08
edges topk 0.3-0.3,4.0,64.35,0.81
nodes random 0.2-0.2 & edges random 0.2-0.2,4.0,43.37,0.90
nodes random 0.3-0.3,4.0,38.53,0.63
